In [ ]:
import $file.^.sparksession

In [ ]:
import sparksession._
import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._
import scala.util.Try
import spark.implicits._

In [ ]:
sealed abstract class PipelineError extends Throwable {
  def msg: String
}

case class ConfigError(msg: String) extends PipelineError
case class ReadError(msg: String) extends PipelineError
case class ExecutionError(msg: String) extends PipelineError
case class TransformationError(msg: String) extends PipelineError

In [ ]:
def goalsPerTeam(input: DataFrame): Either[PipelineError, DataFrame] =
    Try{
        input.select(
            func.explode(
              func.array(
                func.struct(
                  $"home".alias("team"),
                  $"homegoals".alias("goals")
                ),
                func.struct(
                  $"away".alias("team"),
                  $"awayGoals".alias("goals")
                )
              )
            ).alias("value")
        )
        .selectExpr("value.*")
        .groupBy($"team")
        .agg(func.sum($"goals").alias("goals"))
    }.toEither.left.map(e => TransformationError(e.getMessage))

In [ ]:
goalsPerTeam(spark.read.parquet("data/la-liga.parquet"))
    .foreach(_.show)